<a href="https://colab.research.google.com/github/juvi-160/YBI-Internship/blob/main/Wine_Prediction_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WINE** **PREDICTION**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4ee4243c3c811f8405ef40cff0b183a4dd30bcec7355826b0be19cac989e83a4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
import pyspark
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_wine
wine = load_wine(as_frame = True)
wine = wine.frame
wine = spark.createDataFrame(wine)


In [ ]:
wine.show()

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|target|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|     0|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|     0|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

In [ ]:
wine.printSchema()

root
 |-- alcohol: double (nullable = true)
 |-- malic_acid: double (nullable = true)
 |-- ash: double (nullable = true)
 |-- alcalinity_of_ash: double (nullable = true)
 |-- magnesium: double (nullable = true)
 |-- total_phenols: double (nullable = true)
 |-- flavanoids: double (nullable = true)
 |-- nonflavanoid_phenols: double (nullable = true)
 |-- proanthocyanins: double (nullable = true)
 |-- color_intensity: double (nullable = true)
 |-- hue: double (nullable = true)
 |-- od280/od315_of_diluted_wines: double (nullable = true)
 |-- proline: double (nullable = true)
 |-- target: long (nullable = true)



In [ ]:
wine.columns

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline',
 'target']

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
va = VectorAssembler(inputCols = ['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline'], outputCol = 'features')

In [ ]:
modeldata = va.transform(wine)

In [ ]:
modeldata.show()

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+--------------------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|target|            features|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+--------------------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|     0|[14.23,1.71,2.43,...|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|     0|[13.2,1.78,2

In [ ]:
train_data,test_data = modeldata.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(featuresCol='features', labelCol='target')

In [ ]:
dt = dt.fit(train_data)

In [ ]:
pred = dt.transform(test_data)

In [ ]:
pred.groupBy('target','prediction').count().show()

+------+----------+-----+
|target|prediction|count|
+------+----------+-----+
|     0|       1.0|    4|
|     0|       0.0|    9|
|     1|       1.0|   13|
|     2|       2.0|   11|
+------+----------+-----+



In [ ]:
from sklearn.metrics import confusion_matrix
y_org = pred.select('target').collect()
y_pred = pred.select('prediction').collect()
cm = confusion_matrix(y_org,y_pred)
cm

array([[ 9,  4,  0],
       [ 0, 13,  0],
       [ 0,  0, 11]])

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mce = MulticlassClassificationEvaluator(labelCol = 'target', predictionCol = 'prediction')

In [ ]:
mce.evaluate(pred)

0.8892710892710893

In [ ]:
spark.stop()